In [1]:
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import src.dino as dino
# from src import plotting

DOWNSAMPLE_RATIO = 4

def get_value_below(df, value):
    df_with_value = df.where(df==value).dropna(how="all").dropna(axis=1)
    row = df_with_value.index[0]
    column = df_with_value.columns[0]
    return df[column][row+1]


# Generate brightness response images for all inputs
data_dir = Path("training")
for file in tqdm(list((data_dir / "images").iterdir()), desc="generating training set"):
    if not file.suffix in [".exr", ".hdr"]:
        print(f"{file} is not an HDR image, skipping...")
        continue
    export_path = str(data_dir / "brightnesses" / file.stem) + ".npy"
    if Path(export_path).exists():
        print(f"{export_path} already exists")
        brightness = np.load(export_path)
        print(brightness.shape)
        continue
    data_file = str(data_dir / "image_data" / file.stem) + "Data.xls"
    data_pd = pd.read_excel(data_file)
    stated_dyn_range = get_value_below(data_pd, "Dyn. Range")
    max_L = get_value_below(data_pd, "Max.")
    min_L = get_value_below(data_pd, "Min.")
    mean_L = float(get_value_below(data_pd, "Mean"))
    image = dino.read_image(str(file))
    image = dino.resize_image(image, resize_width=round(image.shape[1]/DOWNSAMPLE_RATIO))
    L = dino.fairchild_to_relative_luminance(image)
    # Scale to stated luminance range
    L = (L / np.mean(L)) * mean_L
    actual_dyn_range = round(np.max(L) / np.min(L)) if np.min(L) > 0 else "inf"
    print(f"Stated vs actual dynamic range: {stated_dyn_range} vs {actual_dyn_range} (max: {round(np.max(L))}, min: {round(np.min(L), 1)})")
    print(f"Stated vs actual mean luminance: {mean_L} vs {round(np.mean(L), 1)}")
    brightness = dino.dn_brightness_model(L)
    # L = torch.Tensor(L).to("cuda")
    # brightness = dino.pytorch.dn_brightness_model(L)
    # brightness = brightness.detach().cpu().to_numpy()
    print(brightness.shape)
    print(f"Exporting {export_path}")
    np.save(export_path, brightness)
    # Visualize brightness response
    # plotting.image_pseudocolor_plot(brightness, title=file.stem, cmap="Grays_r", display=True)

generating training set:   0%|          | 0/27 [00:00<?, ?it/s]

Stated vs actual dynamic range: 0.3K:1 vs 5524 (max: 18067, min: 3.299999952316284)
Stated vs actual mean luminance: 1430.0 vs 1430.0
(712, 1071)
Exporting training/brightnesses/MirrorLake.npy
Stated vs actual dynamic range: 0.3K:1 vs 245 (max: 8940, min: 36.5)
Stated vs actual mean luminance: 2570.0 vs 2570.0
(712, 1072)
Exporting training/brightnesses/TupperLake(1).npy
Stated vs actual dynamic range: 3.3K:1 vs 18263 (max: 122299, min: 6.699999809265137)
Stated vs actual mean luminance: 2320.0 vs 2320.0
(712, 1072)
Exporting training/brightnesses/MasonLake(2).npy
Stated vs actual dynamic range: 25K:1 vs 36036 (max: 348529, min: 9.699999809265137)
Stated vs actual mean luminance: 5730.0 vs 5730.0
(603, 1072)
Exporting training/brightnesses/MammothHotSprings.npy
Stated vs actual dynamic range: 15.5K:1 vs 176724 (max: 466585, min: 2.5999999046325684)
Stated vs actual mean luminance: 10500.0 vs 10500.0
(712, 1072)
Exporting training/brightnesses/DevilsTower.npy
Stated vs actual dynamic ra